In [ ]:
#! wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec

In [ ]:
#Getting list of all Categories to consider from nounsFinal.txt
labels = open ('nounsFinal-Updated.txt', 'r+')
lables_list = []
for line in labels.readlines():
    lables_list.append(line.split(' ')[0].rstrip())

In [ ]:
print(len(lables_list))
print(lables_list[1:])

In [ ]:
#Checking if all categories are present in training, validation and testing
import os

data_dir = os.path.join('/data/home/karan1234/notebooks/Karan-Paper','BlobStorage')
validation_data_dir = os.path.join(data_dir, 'validation_data')
train_data_dir = os.path.join(data_dir, 'train_data')
test_data_dir = os.path.join(data_dir,'test_data')
#print(train_data_dir,validation_data_dir,test_data_dir)

train_classes = os.listdir(train_data_dir)
test_classes = os.listdir(test_data_dir)
valid_classes = os.listdir(validation_data_dir)

print(len(train_classes))
#print(train_classes[1:])

print(len(valid_classes))
#print(valid_classes[1:])

print(len(test_classes))
#print(test_classes[1:])

#Finding classes in NounsFinal but not in Train
def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

print('NounsFinal vs Train')
print(Diff(lables_list, train_classes))
print(Diff(train_classes, lables_list))

print('NounsFinal vs Validation')
print(Diff(lables_list, valid_classes))

print('Train vs Validation')
print(Diff(train_classes, valid_classes))

print('Train vs Test')
print(Diff(train_classes, test_classes))

print('NounsFinal vs Test')
print(Diff(lables_list, test_classes))

In [ ]:
# finding categories that may be misspelled
!pip install pyspellchecker
from spellchecker import SpellChecker
spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(lables_list)
print(misspelled)

In [ ]:
#Checking if all categories are present in training, validation and testing
import os

data_dir = os.path.join('/data/home/karan1234/notebooks/Karan-Paper','BlobStorage')
validation_data_dir = os.path.join(data_dir, 'validation_data')
train_data_dir = os.path.join(data_dir, 'train_data')
test_data_dir = os.path.join(data_dir,'test_data')

print(train_data_dir)

import os
total_size = 0
for root, dirs, files in os.walk(train_data_dir):
    for f in files:
        total_size += os.path.getsize(os.path.join(root, f))
print(total_size)

In [ ]:
#Testing 1
#Looking for a particular word from fasttext file
fastext = open('wiki.en.vec', 'r+')
#fastext_dict = {}
counter = 0
for line in fastext:
#    if counter < 5:
    
    splitLine = line.split()
    word = splitLine[0]
    if word in ['book']:
        embedding = [float(el) for el in splitLine[1:]]
        print(word)
        print(embedding)
   # else:
    #    break
    #counter +=1
fastext.close()

In [ ]:
#Testing 2
#Looping through category list and finding from fastext file
#fastext = open('wiki.en.vec', 'r+')
fastext_dict = {}
counter = 0
for lookup_word in lables_list:
    print(lookup_word)
    fastext = open('wiki.en.vec', 'r+')
    for line in fastext:
            splitLine = line.split()
            word = splitLine[0]
            if word == lookup_word:
                print(word+' '+str(counter))
                embedding = [float(el) for el in splitLine[1:]]
                fastext_dict[word] = embedding
                counter += 1
                break
    fastext.close()  

In [ ]:
#Looping through fastext file and finding matches with category list
#This method is faster but gives error at the end for some reason.Check!
fastext = open('wiki.en.vec', 'r+')
fastext_dict = {}
counter = 0
for line in fastext:
        splitLine = line.split()
        word = splitLine[0]
        if word in lables_list:
            print(word+' '+str(counter))
            embedding = [float(el) for el in splitLine[1:]]
            fastext_dict[word] = embedding
            counter += 1
        else:
            continue
fastext.close()    

In [ ]:
#Method from orginal code of Karan
def file_reader(file_name):
    fastext = open(file_name, 'r+')
    line = fastext.readline()
    counter = 0
    #fastext_dict = {}

    while line :
        line = fastext.readline().split(" ")
       # print(line)
        if line[0] in lables_list:
            yield line       
            print(line[0]+' '+counter)
            if counter == 399:
               break
            counter += 1
    fastext.close()

#fastext_dict[line[0]] = [float(el) for el in line[1:-1]]
#fastext_dict = {}
#for line in file_reader('wiki.en.vec'):
    # print(line)
#    fastext_dict[line[0]] = [float(el) for el in line[1:-1]]
    

In [ ]:
print(len(fastext_dict))
#print(fastext_dict.keys())
print(fastext_dict['car'])

#Saving fastext_dict for future use in testing code
import pickle
f = open("fastext_dict.pkl","wb")
pickle.dump(fastext_dict,f)
f.close()

In [2]:
#Reading fastest_dict
import pickle

fastext_dict = pickle.load(open('/data/home/karan1234/notebooks/Karan-Paper/fasttext/fastext_dict.pkl',"rb"))

In [5]:
print(len(fastext_dict))

379


In [3]:
from sklearn.cluster import KMeans
import numpy as np
import math
X = []
keys = []
for k, v in fastext_dict.items():
    #print(type(v[0]))
    X.append(v)
    keys.append(k)
  #  print(k,v)
    
def cosine_similarity(v1, v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

In [7]:
#K-Means
kClusterValues= [i for i in np.arange(10,250,5)]
#kClusterValues = [5,20,35]

f = open("clusterCenters.txt",'w')

for k in kClusterValues:
    
    kmeans = KMeans(n_clusters=k, random_state=0).fit(X)
    #print(kmeans.cluster_centers_)
    #print(kmeans.labels_)
    centroids  = kmeans.cluster_centers_  #means of shape [10,] 
    #for centroid in centroids:
      #  print(centroid)
    uniq_label = list(set(kmeans.labels_))
    keyList = []
    for index_label, label in enumerate(uniq_label):
        maxSim = 0
        maxIndex = -99
        cluster = []
        for index,label1 in enumerate(kmeans.labels_):

            if label == label1:
                cluster.append(keys[index])
                if maxSim < cosine_similarity(X[index],centroids[index_label]):
                   maxSim = cosine_similarity(X[index],centroids[index_label])
                   maxIndex = index

                   #print(maxIndex)
                   #print(cosine_similarity(X[index],centroids[index_label]))
        # print(keys[maxIndex])
        keyList.append(keys[maxIndex])
       # print(cluster)
    strK = "model" + str(k) + "  " + " ".join(keyList) +"\n"
    f.write(strK)
    
f.close()

In [ ]:
#K-Means with selecting K using elbow method
cost =[] 
for i in range(5, 370): 
    KM = KMeans(n_clusters = i, max_iter = 500) 
    KM.fit(X) 
      
    # calculates squared error 
    # for the clustered points 
    cost.append(KM.inertia_)      
  

In [ ]:
#Plotting
%matplotlib inline
import matplotlib.pyplot as plt  
from matplotlib import style 
# plot the cost against K values 
plt.plot(range(5, 370), cost, color ='g', linewidth ='3') 
plt.xlabel("Value of K") 
plt.ylabel("Sqaured Error (Cost)") 
plt.show() # clear the plot 
# the point of the elbow is the  
# most optimal value for choosing k 

In [ ]:
#Dendrogram for Heirarchial clustering
import scipy.cluster.hierarchy as sch
%matplotlib inline
import matplotlib.pyplot as plt 

dendrogram = sch.dendrogram(sch.linkage(X, method  = "ward"))
plt.title('Dendrogram')
plt.xlabel('Classes')
plt.ylabel('Euclidean distances')
plt.show()


In [ ]:
#Choosing k = 7
from sklearn.cluster import AgglomerativeClustering

hclust = AgglomerativeClustering(n_clusters=7, affinity='cosine', linkage='average').fit(X)
uniq_label = list(set(hclust.labels_))
centroids  = hclust.cluster_centers_  
print(uniq_label)
   
keyList = []
for index_label, label in enumerate(uniq_label):
    maxSim = 0
    maxIndex = -99
    cluster = []
    for index,label1 in enumerate(hclust.labels_):

        if label == label1:
            cluster.append(keys[index])
            if maxSim < cosine_similarity(X[index],centroids[index_label]):
                maxSim = cosine_similarity(X[index],centroids[index_label])
                maxIndex = index

                   #print(maxIndex)
                   #print(cosine_similarity(X[index],centroids[index_label]))
        # print(keys[maxIndex])
    keyList.append(keys[maxIndex])
       # print(cluster)
strK = "model" + str(k) + "  " + " ".join(keyList) +"\n"
f.write(strK)


In [ ]:
# the point of the elbow is the  
# most optimal value for choosing k 

In [ ]:
#GMM